In [ ]:
!pip install -q rouge rouge_score bert-score nltk evaluate hazm

In [ ]:
%%capture
!pip install -U datasets 

In [4]:
from evaluate import load
from prettytable import PrettyTable
from rouge import Rouge
import json
from datasets import Dataset 
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, get_scheduler, MT5ForConditionalGeneration, MT5Tokenizer
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
import os
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
import evaluate
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
# from rouge import Rouge

In [ ]:
# import os
# os.listdir('/kaggle/input/persianqa')

In [ ]:
# # Step 1: Download the rouge.py file from GitHub and rename it
# !wget https://raw.githubusercontent.com/Mofid-AI/persian-nlp-benchmark/main/rouge.py -O /kaggle/working/custom_rouge.py

# # Step 2: Add the current directory to the system path and import the necessary class
# import sys
# sys.path.append('/kaggle/working')

# from custom_rouge import RougeScorer

# # # Step 3: Define reference and generated sentences
# # ref = 'سینا امروز سخت کار میکند.'
# # gen = 'سینا و نیما امروز کار نمیکنند.'

# # # Initialize the RougeScorer for rouge1, rouge2, and rougeL
# # rouge_scorer = RougeScorer(['rouge1', 'rouge2', 'rougeL'])

# # # Calculate the scores
# # scores = rouge_scorer.score(ref, gen)

# # # Print the results
# # for rouge_type, score in scores.items():
# #     print(f"{rouge_type}: Precision: {score.precision}, Recall: {score.recall}, F-measure: {score.fmeasure}")

In [6]:
# Load the training data
with open('/kaggle/input/persianqa/pqa_train.json') as f:
    persianqa_train = json.load(f)

# Load the testing data
with open('/kaggle/input/persianqa/pqa_test.json') as f:
    persianqa_test = json.load(f)
    
with open("/kaggle/input/persianquad/train.json") as f:
    persianquad_train = json.load(f)
    
with open("/kaggle/input/persianquad/test.json") as f:
    persianquad_test = json.load(f)

In [7]:
# Function to clean and keep only context and questions
def clean_dataset(data):
    cleaned_data = []

    for item in data['data']:
        # Iterate through paragraphs
        for paragraph in item['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                if not qa['is_impossible']:
                    question = qa['question']
                    cleaned_data.append({
                        'context': context,
                        'question': question
                    })

    return cleaned_data

# Clean both train and test sets
cleaned_persianqa_train = clean_dataset(persianqa_train)
cleaned_persianqa_test = clean_dataset(persianqa_test)
cleaned_persianquad_train = clean_dataset(persianquad_train)
cleaned_persianquad_test = clean_dataset(persianquad_test)

# Save the cleaned datasets
with open('cleaned_pqa_train.json', 'w') as file:
    json.dump(cleaned_persianqa_train, file, ensure_ascii=False, indent=4)

with open('cleaned_pqa_test.json', 'w') as file:
    json.dump(cleaned_persianqa_test, file, ensure_ascii=False, indent=4)
    
with open('cleaned_pquad_train.json', 'w') as file:
    json.dump(cleaned_persianquad_train, file, ensure_ascii=False, indent=4)

with open('cleaned_pquad_test.json', 'w') as file:
    json.dump(cleaned_persianquad_test, file, ensure_ascii=False, indent=4)

print("Datasets cleaned and saved successfully!")

Datasets cleaned and saved successfully!


In [8]:
with open('cleaned_pqa_train.json', 'r') as file:
    qa_train = json.load(file)

with open('cleaned_pqa_test.json', 'r') as file:
    qa_test = json.load(file)
    
with open('cleaned_pquad_train.json', 'r') as file:
    quad_train = json.load(file)
    
with open('cleaned_pquad_test.json', 'r') as file:
    quad_test = json.load(file)

train = qa_train + quad_train
test = qa_test + quad_test
train, val = train_test_split(train, test_size=0.15, random_state=42)

train = train[0:64]
val = val[0:32]
test = test[0:16]

# Convert lists to datasets
train_data = Dataset.from_list(train)
val_data = Dataset.from_list(val)
test_data = Dataset.from_list(test)

In [9]:
train_data

Dataset({
    features: ['context', 'question'],
    num_rows: 64
})

In [ ]:
# max_len_qa = 0
# min_len_qa = 12000
# max_len_quad = 0
# min_len_quad = 12000

# for item in qa_train:
#     if len(item['context'].split()) > max_len_qa:
#         max_len_qa = len(item['context'].split())
#         text = item['context']
        
# for item in qa_test:
#     if len(item['context'].split()) > max_len_qa:
#         max_len_qa = len(item['context'].split())
#         text = item['context']
        
# for item in qa_train:
#     if len(item['context'].split()) < min_len_qa:
#         min_len_qa = len(item['context'].split())
#         text = item['context']

# for item in qa_test:
#     if len(item['context'].split()) < min_len_qa:
#         min_len_qa = len(item['context'].split())
#         text = item['context']
    
# for item in quad_train:
#     if len(item['context'].split()) > max_len_quad:
#         max_len_quad = len(item['context'].split())
#         text = item['context']
        
# for item in quad_test:
#     if len(item['context'].split()) > max_len_quad:
#         max_len_quad = len(item['context'].split())
#         text = item['context']
        
# for item in quad_train:
#     if len(item['context'].split()) < min_len_quad:
#         min_len_quad = len(item['context'].split())
#         text = item['context']

# for item in quad_test:
#     if len(item['context'].split()) < min_len_quad:
#         min_len_quad = len(item['context'].split())
#         text = item['context']
# c=0
# s=0
# for item in quad_train:
#     c+=1
#     s+=len(item['context'].split())
#     avg_quad_train = s/c
    
# c=0
# s=0
# for item in qa_train:
#     c+=1
#     s+=len(item['context'].split())
#     avg_qa_train = s/c
        
# print(max_len_qa)
# print(min_len_qa)
# print(avg_qa_train)
# print(max_len_quad)
# print(min_len_quad)
# print(avg_quad_train)

In [10]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-base", legacy=False, clean_up_tokenization_spaces=True)
# model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-large")

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
from torch.utils.data import Dataset as TDataset

class QuestionGenerationDataset(TDataset):
    def __init__(self, data, tokenizer, max_input_length=360, max_output_length=64):
        self.data_items = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length

    def __len__(self):
        return len(self.data_items)

    def __getitem__(self, idx):
        context = self.data_items[idx]['context']
        question = self.data_items[idx]['question']

        inputs = self.tokenizer(
            context + " </s> سوال: ",
            max_length=self.max_input_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        targets = self.tokenizer(
            question,
            max_length=self.max_output_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        input_ids = inputs.input_ids.squeeze(0)
        attention_mask = inputs.attention_mask.squeeze(0)
        labels = targets.input_ids.squeeze(0)
        labels[labels == self.tokenizer.pad_token_id] = -100
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [12]:
from torch.utils.data import DataLoader

train_dataset = QuestionGenerationDataset(train_data, tokenizer)
val_dataset = QuestionGenerationDataset(val_data, tokenizer)
test_dataset = QuestionGenerationDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True)

In [13]:
def save_model_safely(model):
    output_dir = "./results/checkpoint"
    os.makedirs(output_dir, exist_ok=True)
    for param in model.parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()

    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

In [14]:
%%capture
import torch.optim.lr_scheduler as lr_scheduler

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
num_epochs = 3
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
logging_steps = 500
eval_steps = 1000
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3, verbose=True)
model.to(device)

In [ ]:
loss_table = PrettyTable()
loss_table.field_names = ["Epoch", "Train Loss", "Val Loss"]
loss_log_file = "training_loss_log.txt"

train_losses = []
val_losses = []
best_eval_loss = float('inf')
early_stopping_counter = 0
best_epoch = 0

# Training Loop
loss_table = PrettyTable()
loss_table.field_names = ["Epoch", "Train Loss", "Val Loss", "Learning Rate"]
loss_log_file = "training_loss_log.txt"

train_losses = []
val_losses = []
best_eval_loss = float('inf')
early_stopping_counter = 0
best_epoch = 0

# Training Loop
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    for step, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        optimizer.zero_grad()

        if (step + 1) % logging_steps == 0:
            print(f"Step {step+1}, Loss: {loss.item():.4f}")

    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    print(f"Training loss: {avg_train_loss:.4f}")

    # Evaluation Loop
    model.eval()
    total_eval_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_eval_loss += loss.item()

    avg_eval_loss = total_eval_loss / len(val_loader)
    val_losses.append(avg_eval_loss)
    print(f"Validation loss: {avg_eval_loss:.4f}")

    # Get the learning rate from the optimizer (there could be multiple param groups)
    current_lr = optimizer.param_groups[0]['lr']
    
    # Log the epoch, train loss, val loss, and learning rate in PrettyTable
    loss_table.add_row([epoch + 1, f"{avg_train_loss:.4f}", f"{avg_eval_loss:.4f}", f"{current_lr:.6f}"])

    # Update learning rate based on validation loss
    scheduler.step(avg_eval_loss)

    if avg_eval_loss < best_eval_loss:
        best_eval_loss = avg_eval_loss
        early_stopping_counter = 0
        best_epoch = epoch
        save_model_safely(model)

# Save the PrettyTable to the log file at the end of training
with open(loss_log_file, "a") as f:
    f.write(str(loss_table) + "\n")

In [ ]:
import matplotlib.pyplot as plt
# Plot Training and Validation Loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss', marker='o')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss', marker='o')
plt.title('Training and Validation Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

loss_plot_file = "training_val_loss_plot.png"
plt.savefig(loss_plot_file)

plt.show()
print(f"Training and validation loss diagram saved to {loss_plot_file}")

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from bert_score import score as bert_score
import evaluate
import sys

# Adding the path to custom_rouge.py
sys.path.append('/kaggle/working')

# Import the custom RougeScorer from the renamed file
from custom_rouge import RougeScorer

# Load other metrics
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("bleu")

nltk_smooth = SmoothingFunction().method4

def compute_metrics(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE
    rouge_scorer_obj = RougeScorer(['rouge1', 'rouge2', 'rougeL'])
    rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    for pred, label in zip(preds, labels):
        scores = rouge_scorer_obj.score(label, pred)
        for key in rouge_scores:
            rouge_scores[key] += scores[key].fmeasure
    rouge_scores = {key: value / len(preds) for key, value in rouge_scores.items()}

    # BLEU
    bleu_scores = []
    for pred, label in zip(preds, labels):
        reference = [label.split()]  
        candidate = pred.split()  
        bleu = sentence_bleu(reference, candidate, smoothing_function=nltk_smooth)
        bleu_scores.append(bleu)
    avg_bleu = sum(bleu_scores) / len(bleu_scores)

    # METEOR
    meteor_scores = []
    for pred, label in zip(preds, labels):
        reference_tokens = label.split()  # Tokenize reference
        pred_tokens = pred.split()  # Tokenize prediction
        meteor = meteor_score([reference_tokens], pred_tokens)  # Pass tokenized inputs
        meteor_scores.append(meteor)
        
    avg_meteor = sum(meteor_scores) / len(meteor_scores)
    
    # BERTScore
    P, R, F1 = bert_score(preds, labels, lang="fa", rescale_with_baseline=False)  # Specify the language if necessary
    avg_bert_f1 = F1.mean().item()

    return {
        'bleu': avg_bleu, 
        'meteor': avg_meteor, 
        'rouge1': rouge_scores['rouge1'], 
        'rouge2': rouge_scores['rouge2'], 
        'rougeL': rouge_scores['rougeL'],
        'bertscore_f1': avg_bert_f1
    }

def evaluate_model(model, tokenizer, data_loader):
    model.eval()
    predictions = []
    references = []
    vocab_size = tokenizer.vocab_size

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(model.device)
            attention_mask = batch['attention_mask'].to(model.device)

            if 'labels' in batch:
                labels = batch['labels'].to(model.device)

            # Generate predictions
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)

            # Ensure generated output tokens are within the valid range of the model's vocabulary
            outputs = outputs.clamp(0, vocab_size - 1)
            
            # Decode generated tokens
            preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)

            if 'labels' in batch:
                # Ensure reference labels are clamped before decoding
                labels = labels.clamp(0, vocab_size - 1)
                refs = tokenizer.batch_decode(labels, skip_special_tokens=True)
            else:
                refs = []

            predictions.extend(preds)
            references.extend(refs)
    
    if len(predictions) == 0:
        print('No predictions generated.')
        return {"rouge1": 0, "rouge2": 0, "rougeL": 0, "bleu": 0, "meteor": 0}

    # Compute metrics
    metrics = compute_metrics(predictions, references)
    return metrics


In [ ]:
pretrained_model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base").to(device)
finetuned_dir = f"./results/checkpoint"
# Test the fine-tuned model (assuming the best fine-tuned model is saved in './results/best_checkpoint')
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained(finetuned_dir).to(device)

In [ ]:
import logging
import transformers
import torch

# Set the logging level to WARNING or ERROR to suppress info/debug logs
logging.getLogger('transformers').setLevel(logging.ERROR)  # Suppress transformer messages
logging.getLogger('torch').setLevel(logging.ERROR)

print("Evaluating pre-trained mT5...")
pretrained_metrics = evaluate_model(pretrained_model, tokenizer, test_loader)

print("Evaluating fine-tuned mT5...")
fine_tuned_metrics = evaluate_model(fine_tuned_model, tokenizer, test_loader)

# Create a table with an extra column for BERTScore F1
table = PrettyTable()
table.field_names = ["Model", "BLEU", "METEOR", "ROUGE-1", "ROUGE-2", "ROUGE-L", "BERTScore F1"]

# Add pre-trained mT5 results
table.add_row([
    "Pre-trained mT5",
    f"{pretrained_metrics['bleu']:.4f}",
    f"{pretrained_metrics['meteor']:.4f}",
    f"{pretrained_metrics['rouge1']:.4f}",
    f"{pretrained_metrics['rouge2']:.4f}",
    f"{pretrained_metrics['rougeL']:.4f}",
    f"{pretrained_metrics['bertscore_f1']:.4f}"  # Adding BERTScore F1
])

# Add fine-tuned mT5 results
table.add_row([
    "Fine-tuned mT5",
    f"{fine_tuned_metrics['bleu']:.4f}",
    f"{fine_tuned_metrics['meteor']:.4f}",
    f"{fine_tuned_metrics['rouge1']:.4f}",
    f"{fine_tuned_metrics['rouge2']:.4f}",
    f"{fine_tuned_metrics['rougeL']:.4f}",
    f"{fine_tuned_metrics['bertscore_f1']:.4f}"  # Adding BERTScore F1
])

# Print the table to the console
print(table)

# Save the metrics to a log file
output_file = "model_metrics_log.txt"
with open(output_file, 'w') as f:
    f.write("Model Evaluation Metrics:\n")
    f.write(str(table))

print(f"Metrics saved to {output_file}")


In [ ]:
def save_test_outputs_to_file(model, tokenizer, test_loader, output_file):
    model.eval()

    # Open file to save results
    with open(output_file, 'w') as f:
        # Loop through the test data
        for batch in test_loader:
            inputs = batch['input_ids'].to(model.device)
            attention_mask = batch['attention_mask'].to(model.device)

            # Generate predictions from the model
            with torch.no_grad():
                generated_ids = model.generate(
                    input_ids=inputs,
                    attention_mask=attention_mask,
                    max_length=128, 
                    num_beams=4,   
                    early_stopping=True
                )

            # Decode the input and generated output tokens
            for i, input_id in enumerate(inputs):
                input_text = tokenizer.decode(input_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)

                # Ensure generated_ids are within valid range
                valid_generated_ids = torch.clamp(generated_ids[i], min=0, max=tokenizer.vocab_size - 1).tolist()
                generated_text = tokenizer.decode(valid_generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

                generated_text = generated_text.replace('<extra_id_0>', '').replace('<extra_id_1>', '') 

                # If reference output is available (for evaluation purposes)
                if 'labels' in batch:
                    reference_ids = batch['labels'][i]

                    # Remove padding token IDs before decoding
                    reference_ids = reference_ids[reference_ids != tokenizer.pad_token_id]

                    # Convert to a list and ensure reference_ids are within valid range
                    reference_ids_list = torch.clamp(reference_ids, min=0, max=tokenizer.vocab_size - 1).tolist()

                    # Decode the reference text
                    reference_text = tokenizer.decode(reference_ids_list, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                else:
                    reference_text = "N/A"

                # Write the input, generated output, and reference output to the file
                f.write(f"Input: {input_text}\n")
                f.write(f"Generated Output: {generated_text}\n")
                f.write(f"Reference Output: {reference_text}\n")
                f.write("\n" + "="*50 + "\n\n")  

    print(f"Test outputs saved to {output_file}")

f_output_file = "finetuned_model_test_outputs.txt"
save_test_outputs_to_file(fine_tuned_model, tokenizer, test_loader, f_output_file)

p_output_file = "pretrained_model_test_outputs.txt"
save_test_outputs_to_file(pretrained_model, tokenizer, test_loader, p_output_file)